<a href="https://colab.research.google.com/github/glee1228/006975/blob/master/bag_of_words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Recognition Using Bags of Words

Referenced Code : https://github.com/CyrusChiu/Image-recognition

Porting by glee1228@naver.com

In [0]:
import pandas as pd
import pickle
import numpy as np
import random
import sklearn
from sklearn.cluster import KMeans
import scipy.cluster.vq as vq

## Google Drive Link
for Downloading pkl files

In [2]:
from google.colab import drive
import os

if os.path.exists('/content/gdrive')==False:
    drive.mount('/content/gdrive')
    print('Google Drive is mounted\n')
else:
    print('Google Drive is already mounted\n')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
Google Drive is mounted



In [38]:
os.chdir('/content/gdrive/My Drive')
print('current path ? ',os.getcwd())
print('List of files in the current path :',os.listdir(os.getcwd()))

current path ?  /content/gdrive/My Drive
List of files in the current path : []


## Prepare Cifar-10 Dataset

In [3]:
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

Using TensorFlow backend.


## Opencv Downgrade
3.4.3 version can not be used because of patents on SIFT

In [4]:
! yes | pip3 uninstall opencv-python

! yes | pip3 uninstall opencv-contrib-python

! yes | pip3 install opencv-python==3.4.2.16
 
! yes | pip3 install opencv-contrib-python==3.4.2.16

Uninstalling opencv-python-3.4.2.16:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/cv2/*
    /usr/local/lib/python3.6/dist-packages/opencv_python-3.4.2.16.dist-info/*
Proceed (y/n)?   Successfully uninstalled opencv-python-3.4.2.16
Uninstalling opencv-contrib-python-3.4.2.16:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/opencv_contrib_python-3.4.2.16.dist-info/*
Proceed (y/n)?   Successfully uninstalled opencv-contrib-python-3.4.2.16
  Using cached https://files.pythonhosted.org/packages/fa/7d/5042b668a8ed41d2a80b8c172f5efcd572e3c046c75ae029407e19b7fc68/opencv_python-3.4.2.16-cp36-cp36m-manylinux1_x86_64.whl
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Using cached https://files.pythonhosted.org/packages/08/f1/66330f4042c4fb3b2d77a159db8e8916d9cdecc29bc8c1f56bc7f8a9bec9/opencv_contrib_python-3.4.2.16-cp36-cp36m-manylinux1_x86_64.whl


In [0]:
import cv2

In [6]:
x_train.shape

(50000, 32, 32, 3)

In [7]:
y_train.shape

(50000, 1)

In [0]:
def extract_sift_descriptors(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    return descriptors

def build_codebook(X, voc_size):
    """
    Inupt a list of feature descriptors
    voc_size is the "K" in K-means, k is also called vocabulary size
    Return the codebook/dictionary
    """
    features = np.vstack((descriptor for descriptor in X))
    kmeans = KMeans(n_clusters=voc_size, n_jobs=-1)
    kmeans.fit(features)
    codebook = kmeans.cluster_centers_.squeeze()
    return codebook


def input_vector_encoder(feature, codebook):
    """
    Input all the local feature of the image
    Pooling (encoding) by codebook and return
    """
    code, _ = vq.vq(feature, codebook)
    word_hist, bin_edges = np.histogram(code, bins=range(codebook.shape[0] + 1), normed=True)
    return word_hist

def bootstrap_x_y_resample(x,y, n=None):
    if n == None:
        n = len(x)
    if len(x)!=len(y):
        print('the number of data is not match the number of label')
    resample_i = np.floor(np.random.rand(n) * len(x)).astype(int)
    X_resample = np.array(x)[resample_i]
    Y_resample = np.array(y)[resample_i]
    # print('original label mean:', sum(np.array(y).astype(int)) / len(y))
    # print('resampled label mean:', sum(np.array(Y_resample).astype(int)) / len(Y_resample))
    return X_resample,Y_resample

In [0]:
VOC_SIZE = 100

random.seed(777)

## Bag of Features Process
### 1. feature extraction
SIFT descriptors ... etc
### 2. clustering and build codebook
k-means algorithm .... etc
### 3. Image representation(making the histogram of features)
Vector Quantization Technique
### 4. classifier learning and recognition
SVM , Naive Bayes, ...etc


In [0]:
feature_extraction = False
clustering_and_build_codebook = True
image_representation = True
learning_and_recognition = True
run_all_process = True

In [0]:
if run_all_process:
    feature_extraction = True
    clustering_and_build_codebook = True
    image_representation = True
    learning_and_recognition = True

In [39]:
if feature_extraction:
    # Training
    print("SIFT feature extraction")
    x_train = [extract_sift_descriptors(img) for img in x_train]
    x_test = [extract_sift_descriptors(img) for img in x_test]
    # Remove None in SIFT extraction
    x_train = [each for each in zip(x_train, y_train) if not each[0] is None]
    x_train, y_train = zip(*x_train)
    x_test = [each for each in zip(x_test, y_test) if not each[0] is None]
    x_test, y_test = zip(*x_test)
    print("Train/Test split: {:d}/{:d}".format(len(y_train), len(y_test)))
    with open('./x_train.pkl','wb') as f:
        pickle.dump(x_train, f)
    with open('./x_test.pkl','wb') as f:
        pickle.dump(x_test, f)
    with open('./y_train.pkl', 'wb') as f:
        pickle.dump(y_train, f)
    with open('./y_test.pkl', 'wb') as f:
        pickle.dump(y_test, f)
else:
    with open('./x_train.pkl','rb') as fp:
        x_train = pickle.load(fp)
    with open('./x_test.pkl','rb') as fp:
        x_test = pickle.load(fp)
    with open('./y_train.pkl','rb') as fp:
        y_train = pickle.load(fp)
    with open('./y_test.pkl','rb') as fp:
        y_test = pickle.load(fp)
    print("SIFT features ard loaded")

SIFT feature extraction
Train/Test split: 49904/9976


In [0]:
# class_value, class_count = np.unique(y_train, return_counts=True)
# print('label value :',class_value)
# print('count per label :', class_count)
# print('Bootstrap n={} resampling'.format(n))
# x_train, y_train = bootstrap_x_y_resample(x_train, y_train, n=n)

In [43]:
import multiprocessing

print('the number of CPU using k-means clustering :',multiprocessing.cpu_count())

the number of CPU using k-means clustering : 2


In [0]:
codebook =None
if clustering_and_build_codebook:
    print("Building the codebook, it will take some time")
    codebook = build_codebook(x_train, voc_size=VOC_SIZE)
    with open('./bow_codebook.pkl', 'wb') as f:
        pickle.dump(codebook, f)
else :
    print('Loading the last codebook.')
    with open('./bow_codebook.pkl','rb') as fp:
        codebook = pickle.load(fp)
print('Codebook Shape is ', codebook.shape)

Building the codebook, it will take some time


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  del sys.path[0]


In [0]:
if image_representation:
    print('Approximate data as codeword vectors by Vector Quantization and Making the histogram of data ')
    print("Bag of words encoding")
    x_train = [input_vector_encoder(x, codebook) for x in x_train]
    x_train = np.asarray(x_train)
    x_test = [input_vector_encoder(each, codebook) for each in x_test]
    x_test = np.asarray(x_test)
    with open('./x_train_hist.pkl','wb') as f:
        pickle.dump(x_train, f)
    with open('./x_test_hist.pkl','wb') as f:
        pickle.dump(x_test, f)
else :
    print('Loading the last Image representation')
    with open('./x_train_hist.pkl','rb') as fp:
        x_train = pickle.load(fp)
    with open('./x_test_hist.pkl','rb') as fp:
        x_test = pickle.load(fp)

In [0]:
if learning_and_recognition:
    svm_classifier(x_train, y_train, x_test, y_test)